# Multilayer Perceptron (MLP) for the names dataset 

In [24]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt



In [2]:
words = open('data/names.txt', 'r').read().split()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [3]:
len(words)

32033

## Tokenization

In [4]:
chars = sorted(list(set(''.join(words))))
stringToIndex = {char:index + 1 for index, char in enumerate(chars)}
stringToIndex['.'] = 0
indexToString = {index:char for char, index in stringToIndex.items()}
print(indexToString)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [5]:
# Building the dataset
block_size = 3
X, Y = [], []
for word in words:
    # print(word)
    context = [0] * block_size
    for ch in word + '.':
        ix = stringToIndex[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join(indexToString[i] for i in context), '------->', indexToString[ix])
        context = context[1:] + [ix] # Crop and append
        
X = torch.tensor(X)
Y = torch.tensor(Y)
X.shape

torch.Size([228146, 3])

In [6]:
# Embedding layer
C = torch.randn((27, 2))
emb = C[X]

# Weights to the hidden layer
W1 = torch.randn((6, 100))
b1 = torch.randn((100,))

## 3 ways to reshape the embedding layer

In [7]:
# Not useful, hard coded. What if my block_size changes?
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], dim=1)

tensor([[ 1.0966, -1.4002,  1.0966, -1.4002,  1.0966, -1.4002],
        [ 1.0966, -1.4002,  1.0966, -1.4002,  0.5659, -0.0033],
        [ 1.0966, -1.4002,  0.5659, -0.0033, -0.4381,  1.0622],
        ...,
        [ 0.9959, -0.9085,  0.9959, -0.9085, -1.0437, -0.2549],
        [ 0.9959, -0.9085, -1.0437, -0.2549,  0.9959, -0.9085],
        [-1.0437, -0.2549,  0.9959, -0.9085,  0.4727, -0.0234]])

In [8]:
# Not efficient, we are creating another tensor
torch.cat(torch.unbind(emb, dim = 1), dim = 1)

tensor([[ 1.0966, -1.4002,  1.0966, -1.4002,  1.0966, -1.4002],
        [ 1.0966, -1.4002,  1.0966, -1.4002,  0.5659, -0.0033],
        [ 1.0966, -1.4002,  0.5659, -0.0033, -0.4381,  1.0622],
        ...,
        [ 0.9959, -0.9085,  0.9959, -0.9085, -1.0437, -0.2549],
        [ 0.9959, -0.9085, -1.0437, -0.2549,  0.9959, -0.9085],
        [-1.0437, -0.2549,  0.9959, -0.9085,  0.4727, -0.0234]])

In [9]:
# Efficient way
emb.view(-1, block_size * C.shape[1])

tensor([[ 1.0966, -1.4002,  1.0966, -1.4002,  1.0966, -1.4002],
        [ 1.0966, -1.4002,  1.0966, -1.4002,  0.5659, -0.0033],
        [ 1.0966, -1.4002,  0.5659, -0.0033, -0.4381,  1.0622],
        ...,
        [ 0.9959, -0.9085,  0.9959, -0.9085, -1.0437, -0.2549],
        [ 0.9959, -0.9085, -1.0437, -0.2549,  0.9959, -0.9085],
        [-1.0437, -0.2549,  0.9959, -0.9085,  0.4727, -0.0234]])

In [10]:
h = torch.tanh(emb.view(-1, block_size * C.shape[1])@(W1) + b1)
h.shape

torch.Size([228146, 100])

### Output layer

In [11]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)
logits = h@W2 + b2
counts = logits.exp()
prob = counts/counts.sum(1, keepdim=True)
loss = prob[torch.arange(len(Y)), Y].log().neg().mean()

In [15]:
loss

tensor(17.9100)

In [13]:
prob[torch.arange(prob.shape[0]), Y]

tensor([1.3716e-09, 8.8659e-08, 4.6781e-05,  ..., 1.5520e-08, 1.6692e-13,
        5.1354e-05])

In [14]:
prob.shape[0]

228146

# Rewriting everything:

In [29]:
g = torch.Generator().manual_seed(42)
C = torch.randn((27,2))

# Generating random parameters
W1 = torch.randn((block_size * C.shape[1], 100))
b1 = torch.randn((100,))
W2 = torch.randn((100, 27))
b2 = torch.randn(27)
params = [C, W1, b1, W2, b2]
print("number of parameters: " + str(sum(p.numel() for p in params)))
# Forward pass:

# Embedding layer
emb = C[X]

# First hidden layer
h = torch.tanh(emb.view(-1, block_size * C.shape[1])@W1 + b1)

# Output layer
logits = h@W2 + b2

# Softmax layer (We can use cross_entropy easily. But we are doing it manually)
counts = logits.exp()
prob = counts/counts.sum(1, keepdim=True)
loss = prob[torch.arange(len(Y)), Y].log().neg().mean()
print(loss.item())
# Use cross_entropy
loss2 = F.cross_entropy(logits, Y) 
print(loss2.item())

if loss.item() == loss2.item():
    print("Both losses are the same")
else:
    print("The algorithm is estimating the loss in a different way")

number of parameters: 3481
17.035829544067383
17.03582763671875
The algorithm is estimating the loss in a different way
